In [7]:
import json
import re
from pprint import pprint

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "meta-llama/Llama-2-7b-hf"

In [12]:
# Define the NPC dataset structure
npc_dataset = {
    "train": [
        {"npc1 dialogue": "Hi, I'm Sarah. I just moved into the building yesterday.",
         "npc2 dialogue": "Oh, nice to meet you, Sarah! I'm James. Welcome to the building. How was your move-in?"},
        {"npc1 dialogue": "It was hectic but exciting. Still getting used to the new place.",
         "npc2 dialogue": "Well, if you need anything, feel free to ask. We're all pretty friendly around here."},
        {"npc1 dialogue": "Thanks, James. That's good to know. Have you been living here for long?",
         "npc2 dialogue": "Yeah, I've been here for about a year now. It's a nice community, everyone gets along pretty well."},
        {"npc1 dialogue": "That's great to hear. I'm looking forward to getting to know everyone.",
         "npc2 dialogue": "Definitely. We have a little get-together in the common area every Friday evening. You should come by sometime."},
        {"npc1 dialogue": "So, what's there to do around here?",
         "npc2 dialogue": "Well, there's a nice park nearby where people usually go for walks or picnics. And there are a few cafes and restaurants within walking distance."},
        {"npc1 dialogue": "Sounds lovely. I'll have to check them out. By the way, do you know if there's a gym around here?",
         "npc2 dialogue": "Yeah, there's a gym a couple of blocks away. It's pretty decent, I go there sometimes after work."},
        {"npc1 dialogue": "Great, I'll have to sign up. I'm trying to get back into shape.",
         "npc2 dialogue": "That's awesome. If you want, I can show you around the neighborhood sometime."},
        {"npc1 dialogue": "I'd appreciate that, thanks! Oh, speaking of which, do you have any plans for today?",
         "npc2 dialogue": "Not really, just catching up on some studying. Why?"},
        {"npc1 dialogue": "I was thinking of exploring the neighborhood. Want to join?",
         "npc2 dialogue": "Sounds like a plan. Let me grab my keys."},
        {"npc1 dialogue": "Hey James, do you enjoy reading? I'm always looking for book recommendations.",
         "npc2 dialogue": "Absolutely! I'm a big fan of fantasy novels. Have you read 'The Name of the Wind' by Patrick Rothfuss?"},
        {"npc1 dialogue": "No, but I've heard great things about it. What's it about?",
         "npc2 dialogue": "It's an epic fantasy story with intricate world-building and compelling characters. You should definitely give it a try."},
        {"npc1 dialogue": "Sounds intriguing. I'll add it to my reading list. Thanks for the recommendation!",
         "npc2 dialogue": ""},
        {"npc1 dialogue": "I love cooking! Do you enjoy cooking too, James?",
         "npc2 dialogue": "Yeah, cooking is one of my favorite hobbies. I've been experimenting with different recipes lately."},
        {"npc1 dialogue": "That's awesome. Do you have any favorite dishes you like to cook?",
         "npc2 dialogue": "I really enjoy making homemade pasta from scratch. It's a bit time-consuming, but the end result is so worth it."},
        {"npc1 dialogue": "Homemade pasta sounds delicious. I've always wanted to try making it myself. Do you have any tips?",
         "npc2 dialogue": "Definitely! I can show you some techniques sometime if you're interested."},
        {"npc1 dialogue": "Hey Sarah, have you been keeping up with your fitness routine?",
         "npc2 dialogue": "I've been trying to, but it's been challenging lately. Do you have any tips for staying motivated?"},
        {"npc1 dialogue": "Setting specific goals and tracking your progress can help. I also find that having a workout buddy or joining group classes can be motivating.",
         "npc2 dialogue": "That makes sense. I'll try incorporating those strategies into my routine. Thanks for the advice!"},
        {"npc1 dialogue": "So, what do you do for a living, Sarah?",
         "npc2 dialogue": "I work as a graphic designer. It's pretty creative and I enjoy it most of the time."},
        {"npc1 dialogue": "That's cool. I'm currently studying computer science at the university. It's challenging, but I find it fascinating.",
         "npc2 dialogue": ""},
        {"npc1 dialogue": "Computer science sounds interesting. What made you choose that field?",
         "npc2 dialogue": "I've always been interested in technology and programming. Plus, there are so many opportunities in the tech industry."},
        {"npc1 dialogue": "Hey James, have you done much traveling?",
         "npc2 dialogue": "Yeah, I've been fortunate enough to visit a few countries. My favorite so far has been Japan. The culture and food were amazing."},
        {"npc1 dialogue": "Japan sounds incredible. I've always wanted to go there. What was your favorite part of the trip?",
         "npc2 dialogue": "Definitely exploring the bustling streets of Tokyo. There's so much energy and excitement everywhere you go."},
        {"npc1 dialogue": "That sounds amazing. I'll have to add it to my bucket list.",
         "npc2 dialogue": ""},
        {"npc1 dialogue": "Hey, have you tried that new coffee shop down the street?",
         "npc2 dialogue": "Yeah, I went there yesterday. The coffee was great, and they have a cozy atmosphere."},
        {"npc1 dialogue": "Sounds perfect. I'll have to check it out sometime.",
         "npc2 dialogue": ""},
        {"npc1 dialogue": "I'm thinking of adopting a pet. Do you have any pets?",
         "npc2 dialogue": "Yeah, I have a cat named Luna. She's adorable."},
        {"npc1 dialogue": "Aw, I love cats! They're so independent. How's Luna doing?",
         "npc2 dialogue": "She's doing well. She loves lounging in the sun and chasing after toys."},
        {"npc1 dialogue": "That sounds delightful. Maybe I'll get a cat too.",
         "npc2 dialogue": ""},
        {"npc1 dialogue": "I'm so tired today. I couldn't sleep well last night.",
         "npc2 dialogue": "Oh no, that's rough. Have you tried any relaxation techniques before bed?"},
        {"npc1 dialogue": "Yeah, I've been practicing meditation, but it hasn't been very effective lately.",
         "npc2 dialogue": "Maybe you could try drinking some herbal tea or reading a book before bed. That might help."},
        {"npc1 dialogue": "Thanks, I'll give that a try. Hopefully, it'll help me sleep better tonight.",
         "npc2 dialogue": ""},
        {"npc1 dialogue": "Hey, do you want to grab lunch together later?",
         "npc2 dialogue": "Sure, I'd love to! Any preferences on where to go?"},
        {"npc1 dialogue": "I was thinking of trying out that new Italian restaurant downtown.",
         "npc2 dialogue": "Sounds good to me. I've heard great things about their pasta dishes."},
        {"npc1 dialogue": "Great, let's meet there at 12:30.",
         "npc2 dialogue": ""}
    ],
    "validation": [
         {"npc1 dialogue": "I'm so excited for the concert next weekend!",
         "npc2 dialogue": "Me too! I've been waiting for this for months."},
        {"npc1 dialogue": "Do you know if there will be any food vendors at the venue?",
         "npc2 dialogue": "I'm not sure, but we can always grab dinner beforehand."},
        {"npc1 dialogue": "That's a good idea. I can't wait to dance the night away!",
         "npc2 dialogue": ""},
        {"npc1 dialogue": "Have you been watching any good TV shows lately?",
         "npc2 dialogue": "I just finished binge-watching a crime thriller series. It was so suspenseful!"},
        {"npc1 dialogue": "I love crime shows. Do you have any recommendations?",
         "npc2 dialogue": "You should definitely check out 'Mindhunter' on Netflix. It's brilliant."},
        {"npc1 dialogue": "Thanks, I'll add it to my watchlist. I'm always looking for new shows to watch.",
         "npc2 dialogue": ""},
        {"npc1 dialogue": "I'm thinking of redecorating my apartment. Do you have any interior design tips?",
         "npc2 dialogue": "I'm no expert, but I find that adding plants and colorful accents can liven up a space."},
        {"npc1 dialogue": "That's a great idea. I've been wanting to add more greenery to my place.",
         "npc2 dialogue": ""},
        {"npc1 dialogue": "Hey, have you heard about the new fitness class at the gym?",
         "npc2 dialogue": "Yeah, I saw the flyers. It looks like a fun way to stay in shape."},
        {"npc1 dialogue": "I'm thinking of giving it a try. Want to join me?",
         "npc2 dialogue": "Sure, I'm always up for trying something new!"},
        {"npc1 dialogue": "Awesome, let's sign up for the next class together.",
         "npc2 dialogue": ""},
    ],
    "test": [
        {"npc1 dialogue": "Do you know if there are any community events happening this weekend?",
         "npc2 dialogue": "I think there's a farmer's market in the park on Saturday."},
        {"npc1 dialogue": "That sounds like a lovely way to spend the day. I'll definitely check it out.",
         "npc2 dialogue": ""},
        {"npc1 dialogue": "I'm craving something sweet. Do you want to go get ice cream?",
         "npc2 dialogue": "That sounds delightful! There's a new ice cream parlor that just opened downtown."},
        {"npc1 dialogue": "Let's go there and indulge in some delicious treats.",
         "npc2 dialogue": ""},
        {"npc1 dialogue": "I'm feeling a bit stressed out lately. How do you manage stress?",
         "npc2 dialogue": "I find that taking short breaks throughout the day and practicing mindfulness exercises help."},
        {"npc1 dialogue": "I'll give that a try. Thanks for the advice!",
         "npc2 dialogue": ""},
        {"npc1 dialogue": "Do you have any plans for the weekend?",
         "npc2 dialogue": "I'm thinking of going for a hike on Saturday and having a movie night with friends on Sunday."},
        {"npc1 dialogue": "That sounds like a fun weekend! I might join you for the hike if that's okay.",
         "npc2 dialogue": "Of course, the more the merrier!"},
        {"npc1 dialogue": "Great, I'll bring some snacks for the hike.",
         "npc2 dialogue": ""},
        {"npc1 dialogue": "I've been wanting to learn a new skill. Do you have any suggestions?",
         "npc2 dialogue": "You could try learning a musical instrument or taking up painting."},
        {"npc1 dialogue": "I've always been interested in playing the guitar. Maybe I'll give it a shot.",
         "npc2 dialogue": ""},
        {"npc1 dialogue": "I'm thinking of going on a road trip next month. Do you want to join?",
         "npc2 dialogue": "That sounds like an adventure! Where are you thinking of going?"},
        {"npc1 dialogue": "I haven't decided yet, but I'm open to suggestions.",
         "npc2 dialogue": "We could explore the nearby national parks or visit some historic towns."}
    ]
}




In [45]:

DEFAULT_SYSTEM_PROMPT = """
In a dialogue between two NPCs, NPC1 initiates with a statement. Please generate NPC2's response.
""".strip()


def generate_training_prompt(
    npc1: str, npc2: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    return f"""### Instruction: {system_prompt}

### Input NPC1:
{npc1.strip()}

### Response NPC2:
{npc2}
""".strip()

In [37]:
def generate_text(data_point):

    return {
        "npc1": data_point["npc1 dialogue"],
        "npc2": data_point["npc2 dialogue"],
        "text": generate_training_prompt(data_point["npc1 dialogue"], data_point["npc2 dialogue"]),
    }

In [46]:
example = generate_text(npc_dataset["train"][0])
example

{'npc1': "Hi, I'm Sarah. I just moved into the building yesterday.",
 'npc2': "Oh, nice to meet you, Sarah! I'm James. Welcome to the building. How was your move-in?",
 'text': "### Instruction: In a dialogue between two NPCs, NPC1 initiates with a statement. Please generate NPC2's response.\n\n### Input NPC1:\nHi, I'm Sarah. I just moved into the building yesterday.\n\n### Response NPC2:\nOh, nice to meet you, Sarah! I'm James. Welcome to the building. How was your move-in?"}

In [47]:
"""
dataset["train"] = process_dataset(dataset["train"])
dataset["validation"] = process_dataset(dataset["validation"])
"""

'\ndataset["train"] = process_dataset(dataset["train"])\ndataset["validation"] = process_dataset(dataset["validation"])\n'

In [50]:
example = generate_text(npc_dataset["train"][1])
example

{'npc1': 'It was hectic but exciting. Still getting used to the new place.',
 'npc2': "Well, if you need anything, feel free to ask. We're all pretty friendly around here.",
 'text': "### Instruction: In a dialogue between two NPCs, NPC1 initiates with a statement. Please generate NPC2's response.\n\n### Input NPC1:\nIt was hectic but exciting. Still getting used to the new place.\n\n### Response NPC2:\nWell, if you need anything, feel free to ask. We're all pretty friendly around here."}

In [51]:
def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

In [52]:
model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False

PackageNotFoundError: No package metadata was found for bitsandbytes

In [ ]:
model.config.quantization_config.to_dict()


In [ ]:
lora_r = 16
lora_alpha = 64
lora_dropout = 0.1
lora_target_modules = [
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj",
]


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
OUTPUT_DIR = "experiments"

%load_ext tensorboard
%tensorboard --logdir experiments/runs

In [ ]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
trainer.model

In [ ]:
from peft import AutoPeftModelForCausalLM

trained_model = AutoPeftModelForCausalLM.from_pretrained(
    OUTPUT_DIR,
    low_cpu_mem_usage=True,
)

merged_model = model.merge_and_unload()
merged_model.save_pretrained("merged_model", safe_serialization=True)
tokenizer.save_pretrained("merged_model")

In [ ]:
model = PeftModel.from_pretrained(model, OUTPUT_DIR)
